# Data Science Nigeria 
## Introduction to ML and Deep Learning Course
### Study Guide Practice Notebook

In [2]:
#Import datasets
import sklearn
from sklearn.datasets import load_boston

In [ ]:
data = load_boston() Print a histogram of the quantity to predict: price import matplotlib.pyplot as plt %matplotlib inline plt.style.use('bmh') plt.figure(figsize=(15, 6)) plt.hist(data.target) plt.xlabel('price ($1000s)') plt.ylabel('count')